# 03_compute_bandSNR

In [27]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

In [33]:
# define variables
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]
band = ['delta', 'theta', 'alpha', 'beta', 'gamma']

# set path
pre_root = '/nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids'
post_root = pjoin(pre_root, 'derivatives', 'preproc_meg-mne_mri-fmriprep')

results_pth = '/nfs/e5/studyforrest/forrest_movie_meg/tech_val_results'
if os.path.exists(results_pth) is False:
    os.mkdir(results_pth)

In [29]:
def load_megdata(meg_root, sub, run):
    """
    load meg data
    desp: str, ['raw', 'preproced']
    
    """
    sub_path = BIDSPath(subject=sub, run=run, task='movie', session='movie', root=meg_root)
    meg_data = mne_bids.read_raw_bids(sub_path)
    
    # get data of meg sensors
    ch_name_picks = mne.pick_channels_regexp(meg_data.ch_names, regexp='M[LRZ]...-4503')
    type_picks = mne.pick_types(meg_data.info, meg=True)
    picks= np.intersect1d(ch_name_picks, type_picks)
    data = meg_data.get_data(picks=picks)
    
    # get data during movie playing
    event = mne.find_events(meg_data, stim_channel='UPPT001', min_duration=2/meg_data.info['sfreq'])
    sample = event[:, 0]
    data = data.take(sample, axis=1)
    
    return data

In [30]:
def bandSNR(data, fs):
    """
    Calculate bandSNR for 
    Parameters
    ----------
    data: array
        [n_channels, n_samples]
        
    Returns
    -------
    bandSNR : array
        [n_channels, n_bands]
    """
    
    # remove linear trend
    data_detrend = signal.detrend(data, axis=-1)

    # convert to frequency domain        
    freqs, psd = signal.welch(data_detrend, fs=fs)

    band = [[1, 4], [4, 8], [8, 13], [13, 30], [30, 100]]
    # "delta": 1-4Hz
    # "theta": 4-8Hz
    # "alpha": 8-13Hz
    # "beta": 13-30Hz
    # "gamma": 30-100Hz

    bandSNR = [np.sum(psd[:, (freqs>i[0]) * (freqs<i[1])], axis=-1) / np.sum(psd[:, freqs<0.5*fs], axis=-1) for i in band]
    bandSNR = np.asarray(bandSNR).T
    
    return bandSNR

In [35]:
# compute bandSNR
pre_bandSNR_data = {}
post_bandSNR_data = {}

for sub in sub_list:
    pre_sub = []
    post_sub = []

    # get runlist
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
        
    for run in run_ls:       
        # compute bandSNR for raw meg data
        pre_data = load_megdata(pre_root, sub=sub, run=run)
        pre_bandSNR = bandSNR(pre_data, fs=600)
        pre_sub.append(pre_bandSNR)
        
        # compute bandSNR for preprocessed meg data
        post_data = load_megdata(post_root, sub=sub, run=run)
        post_bandSNR = bandSNR(post_data, fs=600)
        post_sub.append(post_bandSNR)

        print('sub-{0} run-{1} done'.format(sub, run))
    
    pre_bandSNR_data[sub] = pre_sub
    post_bandSNR_data[sub] = post_sub
    

ds directory : /nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.65   74.88    0.00 mm <->    4.65   74.88    0.00 mm (orig :  -65.68   46.24 -249.17 mm) diff =    0.000 mm
      -4.65  -74.88    0.00 mm <->   -4.65  -74.88    0.00 mm (orig :   42.00  -58.24 -250.44 mm) diff =    0.000 mm
      92.94    0.00    0.00 mm <->   92.94   -0.00    0.00 mm (orig :   46.46   62.07 -225.18 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-01_meg.ds/sub-01_ses-movie_task-movie_run-01_meg.meg4: 
    System clock channel is availabl

<ipython-input-29-24e96c24a2e0>:8: RuntimeWarning: Participants file not found for sub-01_ses-movie_task-movie_run-01_meg.fif... Not reading in any particpants.tsv data.
  meg_data = mne_bids.read_raw_bids(sub_path)


22502 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91 255]
sub-01 run-01 done
ds directory : /nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-02_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.38   74.84    0.00 mm <->    4.38   74.84   -0.00 mm (orig :  -66.27   45.00 -250.77 mm) diff =    0.000 mm
      -4.38  -74.84    0.00 mm <->   -4.38  -74.84    0.00 mm (orig :   42.01  -58.72 -249.85 mm) diff =    0.000 mm
      92.81    0.00    0.00 mm <->   92.81   -0.00    0.00 mm (orig :   45.83   

<ipython-input-29-24e96c24a2e0>:8: RuntimeWarning: Participants file not found for sub-01_ses-movie_task-movie_run-02_meg.fif... Not reading in any particpants.tsv data.
  meg_data = mne_bids.read_raw_bids(sub_path)


22602 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91 255]
sub-01 run-02 done
ds directory : /nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-03_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.19   74.86    0.00 mm <->    4.19   74.86   -0.00 mm (orig :  -66.73   44.31 -253.98 mm) diff =    0.000 mm
      -4.19  -74.86    0.00 mm <->   -4.19  -74.86    0.00 mm (orig :   42.69  -58.14 -249.36 mm) diff =    0.000 mm
      92.78    0.00    0.00 mm <->   92.78   -0.00    0.00 mm (orig :   44.87   

<ipython-input-29-24e96c24a2e0>:8: RuntimeWarning: Participants file not found for sub-01_ses-movie_task-movie_run-03_meg.fif... Not reading in any particpants.tsv data.
  meg_data = mne_bids.read_raw_bids(sub_path)


22602 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91 255]
sub-01 run-03 done
ds directory : /nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-04_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.46   75.07    0.00 mm <->    4.46   75.07    0.00 mm (orig :  -66.10   46.10 -255.06 mm) diff =    0.000 mm
      -4.46  -75.07    0.00 mm <->   -4.46  -75.07    0.00 mm (orig :   42.51  -57.85 -250.79 mm) diff =    0.000 mm
      92.52    0.00    0.00 mm <->   92.52    0.00    0.00 mm (orig :   45.04   

<ipython-input-29-24e96c24a2e0>:8: RuntimeWarning: Participants file not found for sub-01_ses-movie_task-movie_run-04_meg.fif... Not reading in any particpants.tsv data.
  meg_data = mne_bids.read_raw_bids(sub_path)


22602 events found
Event IDs: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91 255]
sub-01 run-04 done
ds directory : /nfs/e5/studyforrest/forrest_movie_meg/gump_meg_bids/sub-01/ses-movie/meg/sub-01_ses-movie_task-movie_run-05_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       4.58   75.17    0.00 mm <->    4.58   75.17    0.00 mm (orig :  -64.33   48.49 -257.45 mm) diff =    0.000 mm
      -4.58  -75.17    0.00 mm <->   -4.58  -75.17    0.00 mm (orig :   40.46  -59.66 -254.66 mm) diff =    0.000 mm
      92.70    0.00    0.00 mm <->   92.70    0.00    0.00 mm (orig :   47.18   

KeyboardInterrupt: 

In [ ]:
# save bandSNR
for sub in sub_list[1:]:
    pre_bandSNR_data[sub].append(np.nan)
    post_bandSNR_data[sub].append(np.nan)
pre_df = pd.DataFrame(pre_bandSNR_data, columns=sub_list, index=run_list+['09'])
post_df = pd.DataFrame(post_bandSNR_data, columns=sub_list, index=run_list+['09'])
pre_df.to_pickle(pjoin(results_pth, 'bandSNR_pre.pickle'))
post_df.to_pickle(pjoin(results_pth, 'bandSNR_post.pickle'))